<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/DL_S08_Problemas_FFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S08: Arquitectura de Red FFNN: Problemas prácticos

# Clasificación Binaria:

**Objetivo:** Clasificación de reseñas de películas.

**Input:** Reseñas

**Output:** positiva o negativa.

**Base de datos:** Se usará un dataset de IMDB (Internet Movie Databased) el cual consta de 50000 reseñas altamente polarizadas. Estas reseñas se dividen en un conjunto de 25000 reseñas para entrenamiento  y 25000 reseñas de evaluación, cada una de las cuales consta de un 50% de reseñas positivas y un 50% de reseñas negativas. 

El dataset de IMBD se encuentra Keras. Este dataset ha sido previamente procesado, en ddonde, las reseñas (secuencia de palabras) ha sido transformada en una secuencia de enteros, en donde cada entero corresponde a una palabra específica en un diccionario.

In [ ]:
from tensorflow import keras
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
train_data[0]

A continuación, se realiza un tratamiento de los datos con el fin de convertirlos a vectores de `numpy` que puedan ser usados en una red neuronal. Para mayor información de este paso, consultar el libro del creador de `keras` [Deep learning with python](http://faculty.neu.edu.cn/yury/AAI/Textbook/Deep%20Learning%20with%20Python.pdf)

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000): 
    results = np.zeros((len(sequences), dimension)) # Creates an all-zero matrix of shape (len(sequences), dimension)
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # Sets specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data) # Vectorized training data
x_test = vectorize_sequences(test_data) # Vectorized training data

y_train = np.asarray(train_labels).astype('float32') # Vectorized training labels. 
y_test = np.asarray(test_labels).astype('float32') # Vectorized test labels.

In [ ]:
len(train_data[1])

In [ ]:
x_train[1].shape

Definamos un modelo de dos capas ocultas, con funciones de activacióón **ReLU**, y una capa de salida con una función de activación **Sigmoide**, con el fin de abordar el problema de clasificación binario:

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=4, batch_size=512,validation_data=(x_test,y_test))
results = model.evaluate(x_test, y_test)

In [ ]:
results

In [ ]:
# Predicción del modelo:
model.predict(x_test)

In [ ]:
y_preds=model.predict_classes(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_preds)

# Clasificación Multiclase:

**Objetivo:** Clasificación de un cable de noticias de un dataset de *Reuters*.

**Input:** Noticias.

**Output:** 46 topicos diferentes.

**Base de datos:** Cada topido tiene al menos 10 ejemplos en el set de entrenamiento

In [ ]:
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

A continuación, se realiza un tratamiento de los datos con el fin de convertirlos a vectores de `numpy` que puedan ser usados en una red neuronal. Para mayor información de este paso, consultar el libro del creador de `keras` [Deep learning with python](http://faculty.neu.edu.cn/yury/AAI/Textbook/Deep%20Learning%20with%20Python.pdf)

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

def to_one_hot(labels, dimension=46): 
    results = np.zeros((len(labels), dimension))

    for i, label in enumerate(labels):
        results[i, label] = 1.
        
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [ ]:
train_labels[1]

In [ ]:
one_hot_train_labels[1]

Creemos a continuación una red neuronal con dos capas ocultas (de 64 neuroranas cada una), con funciones de activación **ReLU**, y una capa de salida con una función de activación **softmax**, con 46 neuronas, una por cada tópico:

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
model.fit(partial_x_train, partial_y_train, epochs=9, batch_size=512, validation_data=(x_val, y_val))

In [ ]:
results = model.evaluate(x_test, one_hot_test_labels)

Veamos los resultados de la evaluación del modelo:

In [ ]:
results

Y procesamos a entender los resultados de la prediccióón para una de las instancias del conjunto de evaluación:

In [ ]:
# predicción.
predictions = model.predict(x_test)

In [ ]:
x_test.shape

In [ ]:
predictions[0]

Para la predicción de la instancia 0, se tiene que el vector de salida tiene la siguiente forma:

In [ ]:
predictions[0].shape

El vector tiene 46 entradas, que corresponden a las probabilidades de pertenencia a cada una de las 46 clases. La suma ade ellas debe ser uno:

In [ ]:
np.sum(predictions[0])

La clase más probable a la cual pertenece esta instancia es la clase:

In [ ]:
np.argmax(predictions[0])

In [ ]:
class_pred=model.predict_classes(x_test)

In [ ]:
class_pred.shape

In [ ]:
class_pred[0]

# Regresión: Predicción de los precios de una casa:

Usaremos el dataset de los precios de casas en Bostón, para realizar la predicción.

El objetivo es obtener a la salida de la red neuronal un valor numérico del precio predicho en función de las caracteristicas de la tabla.

Definiremos un arquitectura de red con dos capas ocultas de 64 neuronas, usando una funcióón de activación **ReLu**, en cada una, y una capa de salida de una neurona, con una función de **activación lineal**:

In [ ]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# normalización de los datos
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

#Función para la definición del modelo
from keras import models
from keras import layers
def build_model(): 
  model = models.Sequential() 
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1)) #Función de activación lineal
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])
  return model

In [ ]:
train_data.shape

In [ ]:
# Creación del modelo
modelo = build_model()
modelo.summary()

In [ ]:
modelo.fit(train_data, train_targets, epochs=500, batch_size=16,validation_data=(test_data,test_targets))

In [ ]:
test_mse_score, test_mae_score = modelo.evaluate(test_data, test_targets)

In [ ]:
modelo.evaluate(train_data,train_targets)

In [ ]:
modelo.evaluate(test_data,test_targets)